***HealthGenie – AI-Powered Personalized Health Coach***

**Introduction**
In today's fast-paced world, personalized health guidance is often inaccessible or prohibitively expensive. Generic fitness apps and diet plans fail to account for individual health conditions, goals, and preferences. HealthGenie addresses this gap by leveraging generative AI to create truly personalized health recommendations.

This project demonstrates how generative AI can transform healthcare by providing tailored diet plans, exercise routines, and health tips based on a user's specific health profile and medical conditions. By analyzing health reports and user input, HealthGenie delivers scientifically-grounded recommendations that adapt to individual needs.

**Problem Statement**
Most fitness and health applications provide generic advice that fails to account for:

Individual medical conditions (e.g., diabetes, hypertension)

Personal goals (weight loss, muscle gain)

Dietary preferences and restrictions

Current fitness levels and limitations

This one-size-fits-all approach often leads to poor adherence, suboptimal results, and potential health risks for users with specific medical conditions.

**Solution: HealthGenie**
HealthGenie is an AI health companion that:

Analyzes medical reports and health data

Generates personalized diet and exercise plans

Provides condition-specific health recommendations

Answers health-related questions with medically-grounded information


In [1]:
!pip install google-generativeai pypdf pillow

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GOOGLE_API_KEY")

In [3]:
# HealthGenie: AI Health Companion
# Import necessary libraries
import os
import google.generativeai as genai
import pypdf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from kaggle_secrets import UserSecretsClient
from IPython.display import Markdown, display
import PIL.Image
from io import BytesIO

# ========== SETUP GEMINI API ========== #
# Get API key from Kaggle Secrets
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GOOGLE_API_KEY")  
genai.configure(api_key=api_key)

# Print available models to verify setup
print("Available Gemini models:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

# Set the model name based on available models
MODEL_NAME = "models/gemini-2.0-flash"  # Updated model name
VISION_MODEL_NAME = "models/gemini-1.5-pro"  # Updated vision model name


Available Gemini models:
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
model

**Generative AI Capability #1:** Document Understanding
One of HealthGenie's core capabilities is extracting and interpreting information from medical reports. This demonstrates the document understanding capability of generative AI models.

In [4]:
# ========== DOCUMENT PROCESSING ========== #
def process_health_report(file_path):
    """Extract text from PDF health reports or text files"""
    text = ""
    try:
        with open(file_path, "rb") as f:
            pdf = pypdf.PdfReader(f)
            for page in pdf.pages:
                text += page.extract_text()
    except Exception as e:
        # If it's not a PDF, try reading as text file
        try:
            with open(file_path, "r") as f:
                text = f.read()
        except Exception as e2:
            text = f"Error processing file: {str(e2)}"
    return text

This function demonstrates document understanding by:
1.Accepting PDF health reports as input.

2.Extracting text content from multiple pages.

3.Handling different file formats with graceful error recovery.

In a real-world application, this could be expanded to handle various medical document formats and extract structured information like lab values, diagnoses, and medications.

**Generative AI Capability #2:** Retrieval-Augmented Generation (RAG)
To ensure our health recommendations are scientifically accurate, we implement RAG to ground responses in verified medical guidelines.

In [5]:
# ========== EMBEDDINGS & RAG SETUP ========== #
def create_embeddings(texts):
    """Create embeddings for a list of texts"""
    embeddings = []
    for text in texts:
        result = genai.embed_content(
            model="models/embedding-001",
            content=text,
            task_type="retrieval_document"
        )
        embeddings.append(result['embedding'])
    return embeddings

# Sample medical guidelines for RAG
medical_guidelines = [
    "The American Diabetes Association recommends 150 minutes per week of moderate-intensity aerobic physical activity for diabetics.",
    "Low glycemic index foods are preferred for diabetes management to prevent blood sugar spikes.",
    "Resistance training twice per week can improve insulin sensitivity in type 2 diabetes patients.",
    "For weight loss, a caloric deficit of 500-750 calories per day is recommended for gradual, sustainable results.",
    "High-protein diets (1.2-1.6g/kg body weight) support muscle growth during resistance training programs."
]

# Create embeddings for guidelines
guideline_embeddings = create_embeddings(medical_guidelines)

def find_relevant_guidelines(query, top_k=2):
    """Find most relevant guidelines using embedding similarity"""
    query_embedding = genai.embed_content(
        model="models/embedding-001",
        content=query,
        task_type="retrieval_query"
    )
    
    # Calculate cosine similarity (simplified approach)
    similarities = []
    for guideline_emb in guideline_embeddings:
        # Dot product for similarity
        similarity = sum(a*b for a, b in zip(query_embedding['embedding'], guideline_emb))
        similarities.append(similarity)
    
    # Get indices of top k similar guidelines
    top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:top_k]
    
    return [medical_guidelines[i] for i in top_indices]


**The RAG implementation:**

Creates vector embeddings for medical guidelines using Gemini's embedding model

Computes semantic similarity between user queries and guidelines

Retrieves the most relevant guidelines to ground health recommendations

Ensures responses are aligned with established medical practices

In a production environment, this could be expanded to include thousands of medical guidelines from sources like the American Diabetes Association, American Heart Association, and peer-reviewed medical literature.

**Generative AI Capability #3: Structured Output**
HealthGenie generates structured health plans with clearly defined sections for diet, exercise, and health tips.

In [6]:
# ========== HEALTH PLAN GENERATION ========== #
def generate_health_plan(patient_data):
    """Generate personalized health plan based on patient data"""
    # Get relevant guidelines using RAG
    query = f"Health recommendations for {patient_data.get('Condition', 'general health')}"
    relevant_guidelines = find_relevant_guidelines(query)
    
    # Create prompt with structured output request
    prompt = f"""
    Based on the following patient data and medical guidelines, create a comprehensive 7-day health plan.
    
    PATIENT DATA:
    {patient_data}
    
    RELEVANT MEDICAL GUIDELINES:
    {relevant_guidelines}
    
    Please provide a structured response with the following sections:
    1. DIET PLAN: Include meal suggestions for 7 days with nutritional information
    2. EXERCISE ROUTINE: Recommend appropriate exercises with duration and frequency
    3. HEALTH TIPS: Provide 5 specific tips relevant to the patient's condition
    
    Format the response in a clear, organized manner with sections and bullet points.
    """
    
    # Generate content using updated model name
    model = genai.GenerativeModel(MODEL_NAME)
    response = model.generate_content(prompt)
    
    return response.text


This function demonstrates structured output by:

Specifying the exact format for the health plan (diet, exercise, tips)

Requesting specific details like nutritional information and exercise duration

Ensuring the output is formatted with sections and bullet points for readability

The structured output makes the health plan actionable and easy to follow, improving the user experience.

**Generative AI Capability #4: Agents**
HealthGenie implements an agent architecture to maintain context across multiple interactions and provide personalized responses.

In [7]:
# ========== INTERACTIVE CHAT AGENT ========== #
class HealthGenieAgent:
    def __init__(self):
        self.model = genai.GenerativeModel(MODEL_NAME)
        self.patient_data = {}
        self.history = []
        
    def initialize_patient(self, patient_info):
        """Set initial patient data"""
        self.patient_data = patient_info
        system_prompt = f"""
        You are HealthGenie, an AI health assistant specialized in providing personalized health advice.
        Your goal is to help users with their health goals like muscle gain, weight loss, or managing conditions.
        
        Current patient information:
        {patient_info}
        
        Always be supportive, accurate, and ground your advice in medical science.
        """
        
        # Instead of using chat, we'll use direct generation with history tracking
        response = self.model.generate_content(system_prompt)
        self.history.append({"role": "system", "content": system_prompt})
        self.history.append({"role": "model", "content": response.text})
        
        return "HealthGenie initialized with patient data!"
    
    def ask(self, query):
        """Send a question to the health agent"""
        # Construct conversation context from history
        context = "\n".join([f"{msg['role']}: {msg['content']}" for msg in self.history])
        
        full_prompt = f"{context}\nuser: {query}"
        response = self.model.generate_content(full_prompt)
        
        # Update history
        self.history.append({"role": "user", "content": query})
        self.history.append({"role": "model", "content": response.text})
        
        return response.text
    
    def generate_plan(self):
        """Generate comprehensive health plan"""
        return generate_health_plan(self.patient_data)
    
    def get_chat_history(self):
        """Return formatted chat history"""
        history = []
        for message in self.history:
            history.append(f"**{message['role']}**: {message['content']}")
        return "\n\n".join(history)


The agent implementation:

Maintains conversation history to provide context-aware responses

Stores patient data to personalize recommendations

Provides methods for both structured plan generation and free-form Q&A

Simulates a chat-like experience with role-based messaging

This agent architecture allows HealthGenie to have meaningful multi-turn conversations with users, answering follow-up questions and providing consistent advice based on the user's health profile.

**Demo Setup**
Let's create some synthetic data to demonstrate HealthGenie's capabilities:

In [8]:
# ========== DEMO WITH SYNTHETIC DATA ========== #
# Create sample patient data
sample_patients = [
    {
        "Name": "John Doe",
        "Age": 35,
        "Gender": "Male",
        "Height": "5'10\"",
        "Weight": "190 lbs",
        "Goal": "Muscle gain",
        "Condition": "None",
        "Allergies": "None",
        "Activity Level": "Moderate"
    },
    {
        "Name": "Jane Smith",
        "Age": 42,
        "Gender": "Female",
        "Height": "5'5\"",
        "Weight": "165 lbs",
        "Goal": "Weight loss",
        "Condition": "Type 2 Diabetes",
        "Allergies": "Peanuts",
        "Activity Level": "Sedentary"
    }
]

# Create sample health report text
sample_report = """
MEDICAL REPORT
Patient: Jane Smith
Date: April 10, 2025
Lab Results:
- Fasting Blood Glucose: 142 mg/dL (High)
- HbA1c: 7.3% (High)
- Total Cholesterol: 210 mg/dL (Borderline High)
- Blood Pressure: 138/88 mmHg (Elevated)

Assessment: Patient has Type 2 Diabetes with suboptimal glycemic control.
Recommendation: Dietary modification, increased physical activity, and medication adjustment.
"""

# Save sample report to file
with open("sample_report.txt", "w") as f:
    f.write(sample_report)


**HealthGenie Demonstration**
Now let's run a comprehensive demonstration of HealthGenie's capabilities:

In [9]:
# ========== DEMONSTRATION ========== #
print("=" * 50)
print("HEALTHGENIE AI HEALTH COMPANION DEMO")
print("=" * 50)

# 1. Initialize agent with patient data
print("\n1. INITIALIZING AGENT WITH PATIENT DATA")
agent = HealthGenieAgent()
agent.initialize_patient(sample_patients[1])  # Using diabetic patient
print("Agent initialized!")

# 2. Generate health plan using RAG
print("\n2. GENERATING PERSONALIZED HEALTH PLAN")
health_plan = agent.generate_plan()
display(Markdown(health_plan))

# 3. Interactive Q&A
print("\n3. INTERACTIVE Q&A WITH HEALTHGENIE")
questions = [
    "What exercises are safe for someone with my condition?",
    "Can you suggest some low-glycemic breakfast options?",
    "How often should I check my blood sugar?"
]

for question in questions:
    print(f"\nQ: {question}")
    answer = agent.ask(question)
    display(Markdown(answer))

# 4. Process health report
print("\n4. PROCESSING HEALTH REPORT")
report_text = process_health_report("sample_report.txt")
print("Extracted report text:")
print(report_text)

# 5. Analyze report with Gemini
print("\n5. ANALYZING HEALTH REPORT")
report_analysis = agent.ask(f"Analyze this health report and provide recommendations:\n{report_text}")
display(Markdown(report_analysis))

# 6. Evaluation
print("\n6. EVALUATING PLAN QUALITY")
evaluation_prompt = f"""
Evaluate the quality of this health plan for a 42-year-old female with Type 2 Diabetes:
{health_plan}

Score from 1-5 on:
- Medical accuracy
- Personalization
- Actionability
- Overall quality
"""
evaluation = genai.GenerativeModel(MODEL_NAME).generate_content(evaluation_prompt)
display(Markdown(evaluation.text))

print("\n" + "=" * 50)
print("HEALTHGENIE DEMO COMPLETE")
print("=" * 50)


HEALTHGENIE AI HEALTH COMPANION DEMO

1. INITIALIZING AGENT WITH PATIENT DATA
Agent initialized!

2. GENERATING PERSONALIZED HEALTH PLAN


Okay, here's a comprehensive 7-day health plan tailored for Jane Smith, based on her patient data and the provided medical guidelines.

**1. DIET PLAN:**

This diet plan focuses on portion control, low glycemic index foods, and balanced macronutrients to manage blood sugar levels and promote weight loss. It also avoids peanuts due to her allergy.

**Important Considerations:**

*   **Calorie Goal:** Aim for 1400-1600 calories per day (consider consulting a registered dietitian for a personalized calorie recommendation).
*   **Macronutrient Ratio (approximate):** 40% carbohydrates (primarily complex), 30% protein, 30% healthy fats.
*   **Hydration:** Drink at least 8 glasses of water per day.
*   **Portion Control:** Use measuring cups and food scales to ensure accurate portion sizes.
*   **Read Labels:** Pay attention to serving sizes, carbohydrate content, and added sugars.
*   **Timing of food intake:** Breakfast, lunch, and dinner should be eaten every day, and you may have a snack in between.

**Meal Suggestions (7 Days):**

**(Nutritional information is approximate and may vary based on specific ingredients and preparation methods. Use a food tracking app like MyFitnessPal for precise calculations.)**

**Day 1:**

*   **Breakfast (Approx. 300 calories):** Oatmeal (1/2 cup dry) with berries (1/2 cup) and a sprinkle of almonds (1/4 oz).
*   **Lunch (Approx. 400 calories):**  Large salad with grilled chicken (4 oz), mixed greens, cucumber, tomatoes, bell peppers, and a light vinaigrette dressing (2 tbsp).
*   **Dinner (Approx. 500 calories):** Baked salmon (4 oz) with steamed broccoli (1 cup) and quinoa (1/2 cup cooked).
*   **Snack (Approx. 100 calories):** Greek yogurt (plain, non-fat, 1/2 cup) with a few blueberries.

**Day 2:**

*   **Breakfast (Approx. 320 calories):** Whole-wheat toast (2 slices) with avocado (1/4) and a hard-boiled egg.
*   **Lunch (Approx. 420 calories):** Leftover Baked salmon (4 oz) with steamed broccoli (1 cup) and quinoa (1/2 cup cooked).
*   **Dinner (Approx. 480 calories):** Chicken stir-fry with brown rice (1 cup), mixed vegetables (2 cups), and a low-sodium soy sauce.
*   **Snack (Approx. 100 calories):** Apple slices with 2 tbsp of almond butter (check for peanut contamination).

**Day 3:**

*   **Breakfast (Approx. 310 calories):** Smoothie made with spinach, unsweetened almond milk (1 cup), protein powder (1 scoop), and berries (1/2 cup).
*   **Lunch (Approx. 410 calories):** Turkey breast sandwich (4 oz) on whole-wheat bread with lettuce, tomato, and mustard. Side of carrot sticks.
*   **Dinner (Approx. 520 calories):** Lentil soup (2 cups) with a side of whole-grain crackers (6).
*   **Snack (Approx. 100 calories):** Small handful (1/4 cup) of almonds and walnuts (avoid mixed nuts due to peanut allergy risk).

**Day 4:**

*   **Breakfast (Approx. 330 calories):** Scrambled eggs (2) with spinach and whole-wheat toast (1 slice).
*   **Lunch (Approx. 430 calories):** Large salad with tuna (4 oz, canned in water), mixed greens, cucumber, tomatoes, celery, and a light vinaigrette dressing (2 tbsp).
*   **Dinner (Approx. 470 calories):** Baked chicken breast (4 oz) with roasted sweet potatoes (1 cup) and green beans (1 cup).
*   **Snack (Approx. 100 calories):** Cottage cheese (1/2 cup, low-fat) with a few slices of peach.

**Day 5:**

*   **Breakfast (Approx. 300 calories):** Greek yogurt (plain, non-fat, 1 cup) with berries (1/2 cup) and chia seeds (1 tbsp).
*   **Lunch (Approx. 400 calories):** Leftover Baked chicken breast (4 oz) with roasted sweet potatoes (1 cup) and green beans (1 cup).
*   **Dinner (Approx. 510 calories):** Black bean burgers (2, homemade or store-bought with low sodium) on whole-wheat buns with lettuce and tomato.
*   **Snack (Approx. 100 calories):** Hard-boiled egg.

**Day 6:**

*   **Breakfast (Approx. 320 calories):** Whole-wheat pancakes (2 small) with berries (1/2 cup) and a drizzle of sugar-free syrup.
*   **Lunch (Approx. 420 calories):** Salad with chick peas, mixed greens, cucumber, tomatoes, bell peppers, and a light vinaigrette dressing (2 tbsp).
*   **Dinner (Approx. 490 calories):** Ground turkey (4 oz) and vegetable chili (1.5 cups) with a dollop of plain Greek yogurt (2 tbsp).
*   **Snack (Approx. 100 calories):** Bell pepper strips with hummus (1/4 cup – check label for peanut-free guarantee).

**Day 7:**

*   **Breakfast (Approx. 310 calories):** Oatmeal (1/2 cup dry) with unsweetened applesauce (1/2 cup) and a sprinkle of cinnamon.
*   **Lunch (Approx. 410 calories):** Leftover Ground turkey (4 oz) and vegetable chili (1.5 cups) with a dollop of plain Greek yogurt (2 tbsp).
*   **Dinner (Approx. 530 calories):** Lean beef sirloin (4 oz) with steamed asparagus (1 cup) and brown rice (1/2 cup).
*   **Snack (Approx. 100 calories):** String cheese (low-fat).

**Dietary Notes:**

*   **Fiber:** Emphasize high-fiber foods to help regulate blood sugar.
*   **Limit Processed Foods:** Avoid sugary drinks, processed snacks, and fast food.
*   **Sweeteners:** Use sugar substitutes (e.g., Stevia, erythritol) in moderation.
*   **Cooking Methods:** Bake, grill, steam, or broil food instead of frying.
*   **Variety:** Include a wide variety of vegetables, fruits, and lean protein sources in your diet.
*   **Consult a Dietitian:** A registered dietitian can provide personalized dietary recommendations based on Jane's specific needs and preferences.

**2. EXERCISE ROUTINE:**

Based on the guidelines and the patient's sedentary activity level, this routine aims to gradually increase activity and improve insulin sensitivity.

*   **Aerobic Exercise (Moderate Intensity):** 150 minutes per week, broken down into 30-minute sessions, 5 days a week.
*   **Resistance Training:** Twice per week, on non-consecutive days.
*   **Flexibility/Stretching:** Daily, for 5-10 minutes.

**Detailed Schedule:**

*   **Monday:** 30-minute brisk walk.  Light stretching.
*   **Tuesday:** Resistance Training (see below). Light stretching.
*   **Wednesday:** 30-minute cycling (stationary or outdoors).  Light stretching.
*   **Thursday:** Resistance Training (see below). Light stretching.
*   **Friday:** 30-minute swimming or water aerobics. Light stretching.
*   **Saturday:** 30-minute brisk walk. Light stretching.
*   **Sunday:** Rest or Active Recovery (light yoga, gentle stretching).

**Resistance Training Routine (Twice per week):**

*   **Warm-up (5 minutes):** Light cardio, such as marching in place or arm circles.
*   **Exercises:** Perform 2-3 sets of 10-12 repetitions for each exercise. Choose a weight or resistance that is challenging but allows you to maintain proper form.
    *   **Squats:** (or Chair squats if needed) – targets legs and glutes.
    *   **Push-ups:** (can be done against a wall or on your knees) – targets chest, shoulders, and triceps.
    *   **Dumbbell Rows:** (using light dumbbells or resistance bands) – targets back and biceps.
    *   **Overhead Press:** (using light dumbbells or resistance bands) – targets shoulders.
    *   **Bicep Curls:** (using light dumbbells or resistance bands) – targets biceps.
    *   **Triceps Extensions:** (using light dumbbells or resistance bands) – targets triceps.
*   **Cool-down (5 minutes):** Stretching major muscle groups.

**Important Considerations:**

*   **Start Slowly:** Begin with shorter exercise durations and lower intensity, gradually increasing as fitness improves.
*   **Listen to Your Body:** Rest when needed and avoid pushing through pain.
*   **Consult a Doctor:** Before starting any new exercise program, especially with Type 2 Diabetes, consult with a physician to ensure it is safe and appropriate.
*   **Monitor Blood Sugar:** Check blood sugar levels before, during, and after exercise to understand how activity affects your body.  Carry a source of fast-acting carbohydrate (e.g., glucose tablets) in case of hypoglycemia.
*   **Proper Form:** Focus on maintaining proper form during exercises to prevent injuries. Consider working with a certified personal trainer to learn correct techniques.
*   **Make it Enjoyable:** Choose activities you enjoy to increase adherence.

**3. HEALTH TIPS:**

Here are 5 specific health tips for Jane Smith, relevant to her condition:

*   **Regular Blood Sugar Monitoring:** Consistently check blood sugar levels as recommended by your doctor. Keep a log to track patterns and trends. This helps you understand how food, exercise, and stress impact your glucose levels.
*   **Foot Care:**  Diabetes can damage nerves and reduce blood flow to the feet. Inspect your feet daily for any cuts, blisters, or sores.  Wear comfortable, well-fitting shoes.  See a podiatrist regularly for foot exams.
*   **Stress Management:**  Stress can elevate blood sugar levels. Practice relaxation techniques such as deep breathing, meditation, yoga, or spending time in nature.
*   **Medication Adherence:** Take prescribed medications exactly as directed by your doctor. Do not skip doses or change the dosage without consulting your physician.
*   **Smoking Cessation:** If you smoke, quit. Smoking increases the risk of complications from diabetes, including heart disease, nerve damage, and kidney disease.
*   **Educate Yourself:** Understanding your condition can help you better manage it. Attend diabetes education classes or workshops, and read reliable sources of information about diabetes.
*   **Get Enough Sleep**: Aim for 7-8 hours of quality sleep each night. Poor sleep can worsen insulin resistance and impact blood sugar control. Establish a regular sleep schedule and create a relaxing bedtime routine.

This health plan is designed as a starting point. It's crucial for Jane Smith to consult with her healthcare team (doctor, registered dietitian, certified diabetes educator) to personalize the plan further based on her individual needs, preferences, and progress.



3. INTERACTIVE Q&A WITH HEALTHGENIE

Q: What exercises are safe for someone with my condition?


Okay, Jane, that's an excellent question. Safety is paramount when it comes to exercise and managing Type 2 Diabetes. It's great that you're thinking about this.

Here are some types of exercises that are generally considered safe and beneficial for someone with Type 2 Diabetes and a sedentary starting point, along with important considerations:

**1. Low-Impact Aerobic Exercise:**

*   **Walking:** This is a fantastic starting point. It's easy to do, requires no special equipment, and can be easily incorporated into your daily routine. Start with short walks (10-15 minutes) and gradually increase the duration and intensity. Focus on maintaining a comfortable pace where you can still hold a conversation.
*   **Swimming:** Water exercise is gentle on the joints, making it a great option if you have any joint pain or mobility issues. The buoyancy of water supports your weight, reducing stress on your knees, hips, and ankles. Start with a gentle swim or water aerobics class.
*   **Cycling (stationary or outdoor):** Cycling is another low-impact option that can help improve cardiovascular health and lower blood sugar levels. Start with short rides on a flat surface or a stationary bike with low resistance.
*   **Elliptical Trainer:** This machine provides a low-impact, full-body workout that can be adjusted to your fitness level.

**Important Considerations for Aerobic Exercise:**

*   **Blood Sugar Monitoring:** *Always* check your blood sugar before, during (if exercising for longer than 30 minutes), and after exercise, especially when you are first starting out or making changes to your routine. This will help you understand how your body responds to different types of activity and prevent hypoglycemia (low blood sugar). Have a quick source of carbohydrates (like glucose tablets or a small juice box) readily available in case your blood sugar drops too low.
*   **Foot Care:** Diabetes can sometimes lead to nerve damage in the feet (neuropathy). Inspect your feet daily for any blisters, cuts, or sores, and wear comfortable, supportive shoes.
*   **Hydration:** Drink plenty of water before, during, and after exercise to stay hydrated.
*   **Listen to Your Body:** Pay attention to any pain or discomfort you experience. Stop if you feel dizzy, lightheaded, or short of breath.

**2. Strength Training (Resistance Training):**

*   **Bodyweight Exercises:** These exercises use your own body weight as resistance and are a great way to start building strength. Examples include:
    *   **Chair squats:** Stand up and sit down from a chair repeatedly.
    *   **Wall push-ups:** Stand facing a wall and do push-ups against the wall.
    *   **Arm raises:** Lift your arms out to the side or in front of you.
    *   **Leg raises:** Lift your legs while sitting or lying down.
*   **Resistance Bands:** These are inexpensive and portable and can be used to add resistance to various exercises.
*   **Light Weights:** Start with very light weights (1-2 lbs) and gradually increase the weight as you get stronger. Focus on proper form to avoid injury.

**Important Considerations for Strength Training:**

*   **Proper Form:** Learn the correct form for each exercise to prevent injuries. Consider working with a certified personal trainer, at least initially, to learn proper technique.
*   **Breathing:** Breathe properly during strength training exercises. Exhale during the exertion phase (e.g., when you're pushing up during a push-up) and inhale during the recovery phase.
*   **Avoid Holding Your Breath:** Holding your breath can increase blood pressure.
*   **Start Slowly:** Begin with one set of 10-12 repetitions for each exercise and gradually increase the number of sets and repetitions as you get stronger.

**3. Flexibility and Balance Exercises:**

*   **Stretching:** Gentle stretching can improve flexibility and range of motion. Hold each stretch for 20-30 seconds. Focus on stretching major muscle groups, such as your legs, arms, and back.
*   **Yoga and Tai Chi:** These practices combine gentle movements, stretching, and relaxation techniques. They can help improve balance, flexibility, and stress levels.
*   **Balance Exercises:** Simple balance exercises, such as standing on one foot for a few seconds at a time, can help improve stability and prevent falls.

**General Recommendations:**

*   **Warm-up:** Always warm up before exercise with 5-10 minutes of light cardio and stretching.
*   **Cool-down:** Cool down after exercise with 5-10 minutes of stretching.
*   **Consistency is Key:** Aim for at least 30 minutes of moderate-intensity exercise most days of the week. Even short bursts of activity throughout the day can be beneficial.
*   **Listen to Your Body:** Rest when you need to, and don't push yourself too hard, especially when you're first starting out.
*   **Medical ID:** Consider wearing a medical ID bracelet or necklace that indicates you have Type 2 Diabetes.

**Crucial Reminder:**

*   **Consult Your Doctor:** Before starting any new exercise program, it is essential to consult with your doctor. They can assess your overall health, evaluate any potential risks, and provide personalized recommendations based on your individual needs and condition. They can also advise you on how to adjust your medication based on your activity level.

I know this is a lot of information, Jane, but I want you to feel empowered and safe as you begin your exercise journey. Which of these options feels most appealing or accessible to you to start with? Perhaps we can explore that in more detail.



Q: Can you suggest some low-glycemic breakfast options?


...


4. PROCESSING HEALTH REPORT
Extracted report text:

MEDICAL REPORT
Patient: Jane Smith
Date: April 10, 2025
Lab Results:
- Fasting Blood Glucose: 142 mg/dL (High)
- HbA1c: 7.3% (High)
- Total Cholesterol: 210 mg/dL (Borderline High)
- Blood Pressure: 138/88 mmHg (Elevated)

Assessment: Patient has Type 2 Diabetes with suboptimal glycemic control.
Recommendation: Dietary modification, increased physical activity, and medication adjustment.


5. ANALYZING HEALTH REPORT


Okay, Jane, let's break down this health report and discuss what it means for you and our plan. It's important to note that I am an AI and cannot provide medical diagnoses or treatment advice. This analysis is for informational purposes only and should be discussed with your doctor for appropriate medical management.

Here's an analysis of your health report:

**Key Findings:**

*   **Fasting Blood Glucose: 142 mg/dL (High):** This indicates that your blood sugar levels are elevated even after an overnight fast. A normal fasting blood glucose level is typically below 100 mg/dL. This result suggests that your diabetes is not well-controlled.
*   **HbA1c: 7.3% (High):** HbA1c is a measure of your average blood sugar levels over the past 2-3 months. A normal HbA1c level for people without diabetes is below 5.7%. The American Diabetes Association generally recommends an HbA1c target of less than 7% for people with diabetes. Your result of 7.3% indicates that your blood sugar levels have been consistently elevated over the past few months, and again, your diabetes management requires adjustment.
*   **Total Cholesterol: 210 mg/dL (Borderline High):** A normal total cholesterol level is typically below 200 mg/dL. Elevated cholesterol levels can increase your risk of heart disease, which is already a concern for people with diabetes.
*   **Blood Pressure: 138/88 mmHg (Elevated):** This indicates that your blood pressure is elevated. A normal blood pressure is typically below 120/80 mmHg. Elevated blood pressure, also known as hypertension, can increase your risk of heart disease, stroke, and kidney disease. The current guideline defines Stage 1 Hypertension as 130-139/80-89 mmHg.

**Overall Assessment:**

The report confirms that you have Type 2 Diabetes with suboptimal glycemic control. This means that your blood sugar levels are consistently above the target range. You also have borderline high cholesterol and elevated blood pressure, which are additional risk factors for heart disease.

**Recommendations from the Report:**

The report recommends dietary modification, increased physical activity, and medication adjustment. These are all important components of diabetes management, and we've already begun to address the first two in our discussions.

**My Recommendations Based on the Report and Our Previous Discussions:**

1.  **Schedule an Urgent Appointment with Your Doctor:** This report underscores the *urgent need* to consult with your doctor as soon as possible. Your doctor will need to review these results, conduct a physical exam, and make appropriate adjustments to your medication regimen. *Do not make any changes to your medication without consulting your doctor.*
2.  **Dietary Modifications (Review and Refine):**
    *   **Re-emphasize low-glycemic eating:** Let's revisit our discussion about low-glycemic breakfast options and expand that to lunch and dinner. Focus on non-starchy vegetables, lean protein, and healthy fats.
    *   **Consider Carbohydrate Counting:** Work with a registered dietitian to learn how to count carbohydrates and adjust your insulin dosage (if applicable) based on your carbohydrate intake.
    *   **Limit Saturated and Trans Fats:** Given your elevated cholesterol levels, it's important to limit your intake of saturated and trans fats, which can raise LDL ("bad") cholesterol.
    *   **Increase Fiber Intake:** Fiber can help lower cholesterol and improve blood sugar control.
    *   **Reduce Sodium Intake:** Given your elevated blood pressure, it's important to reduce your sodium intake. Avoid processed foods and limit your use of salt when cooking.
3.  **Increased Physical Activity (Reinforce):**
    *   **Gradual Progression:** Continue to gradually increase the intensity and duration of your workouts, as we discussed.
    *   **Variety:** Incorporate a variety of activities to stay motivated and prevent boredom.
    *   **Consistency:** Aim for at least 150 minutes of moderate-intensity aerobic activity per week, as well as strength training exercises at least two days per week.
    *   **Blood Sugar Monitoring:** Closely monitor your blood sugar levels before, during, and after exercise to see how your body responds and prevent hypoglycemia.
4.  **Medication Adjustment (Doctor's Guidance):**
    *   **Discuss Medication Options with Your Doctor:** Your doctor may need to adjust your current medications or add new medications to help you achieve better blood sugar control. They may also consider prescribing medications to lower your cholesterol and blood pressure.
    *   **Adhere to Your Medication Regimen:** Take your medications exactly as prescribed by your doctor.
5.  **Weight Loss (Continue to Prioritize):**
    *   **Sustainable Approach:** Focus on making sustainable lifestyle changes that you can maintain over the long term.
    *   **Calorie Deficit:** Create a calorie deficit through a combination of dietary modifications and increased physical activity.
6.  **Stress Management (Implement):**
    *   **Stress-Reducing Activities:** Incorporate stress-reducing activities into your daily routine, such as yoga, meditation, or deep breathing exercises.
7.  **Regular Monitoring:**
    *   **Blood Sugar Monitoring:** Continue to monitor your blood sugar levels as recommended by your doctor.
    *   **Regular Check-ups:** Schedule regular check-ups with your doctor to monitor your progress and make any necessary adjustments to your treatment plan.
8.  **Consult a Specialist:**
    *   **Registered Dietitian:** A registered dietitian can help you create a personalized meal plan that meets your individual needs and preferences.
    *   **Certified Diabetes Educator (CDE):** A CDE can provide you with education and support to help you manage your diabetes effectively.

**Key Takeaways:**

This report highlights the importance of taking a proactive approach to managing your diabetes and overall health. It's crucial to work closely with your doctor and other healthcare professionals to develop a comprehensive treatment plan that addresses all of your risk factors. The lifestyle changes we've been discussing are a good foundation, but the report clearly indicates the need for medical intervention and likely medication adjustments.

Again, please schedule an appointment with your doctor as soon as possible to discuss these results and develop a plan of action.

What specific questions do you have about this report or the recommendations I've provided?



6. EVALUATING PLAN QUALITY


Here's an evaluation of the provided health plan, broken down by the requested categories:

**Overall Score: 4.3/5**

Here's the breakdown:

*   **Medical Accuracy: 5/5**

    *   **Strengths:** The plan reflects current medical guidelines for managing Type 2 Diabetes. It emphasizes key principles like:
        *   Low glycemic index foods
        *   Balanced macronutrients
        *   Regular blood sugar monitoring
        *   Importance of exercise (both aerobic and resistance)
        *   Foot care for diabetics
        *   Stress management
        *   Medication adherence
    *   **Areas for Improvement:** None - it gets key facts and standards right.

*   **Personalization: 4/5**

    *   **Strengths:**
        *   The plan considers the person is a 42-year-old female.
        *   It acknowledges the Type 2 Diabetes diagnosis.
        *   It takes into account the peanut allergy.
        *   It notes a previous sedentary lifestyle and proposes a gradual increase in activity.
        *   Mentions need for consultation with dietician and doctor.
    *   **Areas for Improvement:**
        *   The calorie goal (1400-1600) is a reasonable starting point but should *absolutely* be individualized after consultation with a dietitian. There's no mention of her current weight, height, or activity level beyond "sedentary," which are all important factors.
        *   The exercise plan, while good in principle, lacks personalization.  There's no mention of existing injuries, limitations, or preferences.
        *   It would be better if a more specific patient history was mentioned for even better personalization.

*   **Actionability: 4.5/5**

    *   **Strengths:**
        *   The meal plans are specific and provide concrete examples of what to eat for each meal and snack.
        *   The exercise routine is broken down into a weekly schedule with clear instructions.
        *   Health tips are practical and easily understandable.
        *   The plan emphasizes the importance of monitoring blood sugar.
    *   **Areas for Improvement:**
        *   While the meal plans are specific, they might be overwhelming for someone just starting.  A simpler "swap" approach (e.g., "Instead of white bread, choose whole-wheat bread") might be more sustainable initially.
        *   Providing a list of resources (e.g., reputable websites, diabetes education programs) would increase actionability.
        *   Specify examples of what kind of exercise would not be good for the patient.

*   **Overall Quality: 4.3/5**

    *   **Strengths:** The plan is well-organized, comprehensive, and medically sound. It covers key aspects of diabetes management (diet, exercise, lifestyle). It provides a good starting point for someone newly diagnosed or looking to improve their diabetes control.
    *   **Areas for Improvement:** The biggest opportunity for improvement is to emphasize the *crucial* need for professional guidance (doctor, dietitian, certified diabetes educator) to tailor the plan to the individual. The plan does mention this, but it should be highlighted more strongly. It also needs better details on possible problems that can occur and ways to avoid them.



HEALTHGENIE DEMO COMPLETE


**Generative AI Capabilities Summary**
HealthGenie demonstrates four key generative AI capabilities:

**1.Document Understanding**

Extracts and interprets information from medical reports.

Processes unstructured text to identify key health metrics.

Enables personalization based on medical documentation.

**2.Retrieval-Augmented Generation (RAG)**

Grounds health recommendations in verified medical guidelines.

Uses semantic search to find relevant health information.

Ensures responses align with established medical practices.

**3.Structured Output**

Generates organized health plans with clear sections.

Provides specific, actionable recommendations.

Formats information for optimal readability and usability.

**4.Agents**

Maintains conversation context across multiple interactions.

Personalizes responses based on user profile.

Enables natural, multi-turn conversations about health topics.

**Real-World Applications**
HealthGenie demonstrates how generative AI can transform healthcare by:

**1.Democratizing Access:** Providing personalized health guidance at scale, making expert-level advice more accessible.

**2.Supporting Healthcare Providers:** Assisting doctors and nutritionists with personalized plan generation, freeing their time for complex cases.

**3.Improving Health Outcomes:** Increasing adherence through personalized recommendations that account for individual conditions and preferences.

**4.Empowering Self-Management:** Helping patients with chronic conditions better understand and manage their health with condition-specific guidance.

**Future Enhancements**
HealthGenie could be extended with:

**1.Multimodal Inputs:** Processing food images, exercise videos, and wearable device data.

**2.Function Calling:** Integrating with fitness trackers, nutrition databases, and medication reminder systems.

**3.Expanded RAG:** Incorporating larger medical knowledge bases and real-time medical research.

**4.Personalized Learning:** Adapting recommendations based on user feedback and progress tracking.

**Conclusion**
HealthGenie demonstrates the transformative potential of generative AI in healthcare. By combining document understanding, RAG, structured output, and agent capabilities, we've created a system that provides personalized, scientifically-grounded health guidance.

@misc{gen-ai-intensive-course-capstone-2025q1,
    author = {Addison Howard and Brenda Flynn and Kinjal Parekh and Myles O'Neill and Nate and Polong Lin},
    title = {Gen AI Intensive Course Capstone 2025Q1},
    year = {2025},
    howpublished = {\url{https://kaggle.com/competitions/gen-ai-intensive-course-capstone-2025q1}},
    note = {Kaggle}
}